In [10]:
#get file paths from user
p1 =readline(STDIN)
p2= readline(STDIN)

function getMetrics(path)
    #open file and read contents into array of words
    f=open(path)
    stuff = readstring(f)
    words = split(stuff)

    #counters for words, sentences
    numWords=0
    numSentences=0
    wordLens=0 #keep track of characters
    wordcounts=Dict() #keep track of counts of each words

    stopwords = ("ourselves", "hers", "between", "yourself", "but", "again", "there", "about", "once", "during", "out", "very", "having", "with", "they", "own", "an", "be", "some", "for", "do", "its", "yours", "such", "into", "of", "most", "itself", "other", "off", "is", "s", "am", "or", "who", "as", "from", "him", "each", "the", "themselves", "until", "below", "are", "we", "these", "your", "his", "through", "don", "nor", "me", "were", "her", "more", "himself", "this", "down", "should", "our", "their", "while", "above", "both", "up", "to", "ours", "had", "she", "all", "no", "when", "at", "any", "before", "them", "same", "and", "been", "have", "in", "will", "on", "does", "yourselves", "then", "that", "because", "what", "over", "why", "so", "can", "did", "not", "now", "under", "he", "you", "herself", "has", "just", "where", "too", "only", "myself", "which", "those", "i", "after", "few", "whom", "t", "being", "if", "theirs", "my", "against", "a", "by", "doing", "it", "how", "further", "was", "here", "than")

    #loop through each word in the book
    for word in words
        numWords=numWords+1
        #if word ends in .!? its the end of a sentence
        if(word[endof(word)]=='!' || word[endof(word)]=='.' || word[endof(word)]=='?')
            numSentences=numSentences+1
        end
        #strip punctuation off of word and add it to dictionary
        sword=lowercase(strip(word, ['.','!','?','‘','\"','“']))
        wordLens+=length(sword)
        if(sword in stopwords)
            continue
        end
        if(haskey(wordcounts, sword))
            wordcounts[sword]=wordcounts[sword]+1
        else
            wordcounts[sword]=1
        end
    end

    #sort the counts by the values (high to low)
    orderedcounts=sort(collect(wordcounts), by = tuple -> last(tuple), rev=true)
    #print results
    println(path)
    println(numWords, " words")
    println(numSentences, " sentences")
    println(wordLens/numWords, " avg word length")
    top=[]
    for i in 1:10
       println(orderedcounts[i][1], " ", orderedcounts[i][2])
        push(top,orderedcounts[i][1])
    end

    close(f)
    return (numWords, numSentences, numWords/numSentences, wordLens/numWords, top)

end

getMetrics(p1)
getMetrics(p2)

STDIN> alice_in_wonderland.txt
STDIN> man_from_mars.txt
29465 words
1083 sentences
4.407330731376209 avg word length
said 425
alice 278
little 122
project 86
one 85
would 81
went 81
alice, 76
like 75
could 71
59441 words
1886 sentences
4.9033831866893225 avg word length
upon 223
one 201
among 147
life 119
labor 109
us 107
many 99
may 95
without 89
project 87
